# **Pre-processamento da MIMIC**
## O objetivo dessa etapa é a serialização dos dados para podermos usar no LSTM, isso significa que:
- Cada paciente/internação tem uma série temporal de até 1.510 pontos (janelas)
- Cada janela tem os atributos médicos
- Formato ideal para LSTM: [pacientes, janelas_tempo, features]
## Ou seja, vamos precisar: 
- agrupar por *subject_id* e *stay_id*
- Ordenar por *janela_index*
- transformar em senquências temporais.

## 1. Importações

In [1]:
import pandas as pd
import numpy as np
# from sklearn.model_selection import train_test_split

## 2. Carregando e visualizando os dados em formato tabular

In [21]:
dataset_orig = pd.read_parquet('D:\HD_Externo\LSTM_apli_MIMIC\mimic')
dataset = dataset_orig.copy()
dataset.head(20)

,subject_id,stay_id,janela_index,inicio_janela,charttime,tem_sepse,fc,pas,pad,pam,fr,spo,tem,ida,pes,alt
0,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:11:00,0.0,NaN,84.0,48.0,56.0,NaN,NaN,NaN,52.0,39.400002,152.0
1,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:12:00,0.0,91.0,NaN,NaN,NaN,24.0,NaN,NaN,52.0,39.400002,152.0
2,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:13:00,0.0,NaN,NaN,NaN,NaN,NaN,98.0,NaN,52.0,39.400002,152.0
3,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:30:00,0.0,93.0,95.0,59.0,67.0,21.0,97.0,NaN,52.0,39.400002,152.0
4,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 15:00:00,0.0,94.0,88.0,56.0,64.0,23.0,97.0,NaN,52.0,39.400002,152.0
5,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 16:00:00,0.0,105.0,NaN,NaN,NaN,21.0,94.0,NaN,52.0,39.400002,152.0
6,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 16:01:00,0.0,NaN,91.0,55.0,64.0,NaN,NaN,NaN,52.0,39.400002,152.0
7,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 17:00:00,0.0,97.0,95.0,58.0,67.0,20.0,95.0,NaN,52.0,39.400002,152.0
8,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 18:00:00,0.0,100.0,86.0,53.0,60.0,21.0,95.0,NaN,52.0,39.400002,152.0
9,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 19:00:00,0.0,97.0,93.0,41.0,56.0,16.0,98.0,NaN,52.0,39.400002,152.0


In [17]:
# 1. Forward fill dentro da janela, limitado a 2 aparições
dataset['fc'] = dataset.groupby(['subject_id', 'janela_index'])['fc'].transform(lambda x: x.fillna(method='ffill', limit=2))

# 2. Preenche o que sobrou com a média da janela
dataset['fc'] = dataset.groupby(['subject_id', 'janela_index'])['fc'].transform(lambda x: x.fillna(x.mean()))


C:\Users\thiag\AppData\Local\Temp\ipykernel_4820\422548764.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset['fc'] = dataset.groupby(['subject_id', 'janela_index'])['fc'].transform(lambda x: x.fillna(method='ffill', limit=2))


In [22]:
colunas_para_tratar = [
    'tem_sepse', 'fc', 'pas', 'pad', 'pam', 'fr',
    'spo', 'tem', 'ida', 'pes', 'alt'
]

for col in colunas_para_tratar:
    if dataset[col].isna().any():
        # 1. Forward fill limitado dentro da janela
        dataset[col] = dataset.groupby(['subject_id', 'janela_index'])[col].transform(
            lambda x: x.fillna(method='ffill', limit=2)
        )
        
        # 2. Preenchimento com a média da janela
        dataset[col] = dataset.groupby(['subject_id', 'janela_index'])[col].transform(
            lambda x: x.fillna(x.mean())
        )
        
        # 3. Preenchimento com a média do paciente
        dataset[col] = dataset.groupby('subject_id')[col].transform(
            lambda x: x.fillna(x.mean())
        )
dataset.head(20)

C:\Users\thiag\AppData\Local\Temp\ipykernel_4820\3079434285.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.fillna(method='ffill', limit=2)


,subject_id,stay_id,janela_index,inicio_janela,charttime,tem_sepse,fc,pas,pad,pam,fr,spo,tem,ida,pes,alt
0,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:11:00,0.0,92.250000,84.000000,48.00,56.000000,23.000000,97.333333,NaN,52.0,39.400002,152.0
1,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:12:00,0.0,91.000000,84.000000,48.00,56.000000,24.000000,97.333333,NaN,52.0,39.400002,152.0
2,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:13:00,0.0,91.000000,84.000000,48.00,56.000000,24.000000,98.000000,NaN,52.0,39.400002,152.0
3,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:30:00,0.0,93.000000,95.000000,59.00,67.000000,21.000000,97.000000,NaN,52.0,39.400002,152.0
4,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 15:00:00,0.0,94.000000,88.000000,56.00,64.000000,23.000000,97.000000,NaN,52.0,39.400002,152.0
5,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 16:00:00,0.0,105.000000,91.250000,51.75,61.750000,21.000000,94.000000,NaN,52.0,39.400002,152.0
6,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 16:01:00,0.0,105.000000,91.000000,55.00,64.000000,21.000000,94.000000,NaN,52.0,39.400002,152.0
7,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 17:00:00,0.0,97.000000,95.000000,58.00,67.000000,20.000000,95.000000,NaN,52.0,39.400002,152.0
8,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 18:00:00,0.0,100.000000,86.000000,53.00,60.000000,21.000000,95.000000,NaN,52.0,39.400002,152.0
9,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 19:00:00,0.0,97.000000,93.000000,41.00,56.000000,16.000000,98.000000,NaN,52.0,39.400002,152.0


In [24]:
dataset.isnull().sum()

subject_id             0
stay_id                0
janela_index           0
inicio_janela          0
charttime              0
tem_sepse              0
fc                   384
pas                43668
pad                44583
pam                43281
fr                  2521
spo                 1080
tem              5637772
ida                32697
pes               196727
alt              2319083
dtype: int64

In [ ]:
dataset_trat = dataset.drop(['tem','alt'], axis=1)
dataset_trat.isnull().sum()

subject_id            0
stay_id               0
janela_index          0
inicio_janela         0
charttime             0
tem_sepse             0
fc                  384
pas               43668
pad               44583
pam               43281
fr                 2521
spo                1080
ida               32697
pes              196727
dtype: int64

In [34]:
print(dataset_trat['pes'].mean())
dataset_trat['pes'] = dataset_trat['pes'].fillna(dataset_trat['pes'].mean())
print(dataset_trat.isnull().sum())

nan_pct = dataset_trat.isna().any(axis=1).mean() * 100
print(f"{nan_pct:.2f}% das linhas têm pelo menos um NaN")

dataset_final = dataset_trat.dropna()
nan_pct = dataset_final.isna().any(axis=1).mean() * 100
print(f"{nan_pct:.2f}% das linhas têm pelo menos um NaN")

82.95795956009374
subject_id           0
stay_id              0
janela_index         0
inicio_janela        0
charttime            0
tem_sepse            0
fc                 384
pas              43668
pad              44583
pam              43281
fr                2521
spo               1080
ida              32697
pes                  0
dtype: int64
1.15% das linhas têm pelo menos um NaN
0.00% das linhas têm pelo menos um NaN


In [3]:
dataset_final = pd.read_parquet('D:\HD_Externo\LSTM_apli_MIMIC\mimic_tratada')
dataset_final.head(20)

,subject_id,stay_id,janela_index,inicio_janela,charttime,tem_sepse,fc,pas,pad,pam,fr,spo,ida,pes
0,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:11:00,0.0,92.250000,84.000000,48.00,56.000000,23.000000,97.333333,52.0,39.400002
1,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:12:00,0.0,91.000000,84.000000,48.00,56.000000,24.000000,97.333333,52.0,39.400002
2,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:13:00,0.0,91.000000,84.000000,48.00,56.000000,24.000000,98.000000,52.0,39.400002
3,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 14:30:00,0.0,93.000000,95.000000,59.00,67.000000,21.000000,97.000000,52.0,39.400002
4,10000032,39553978,0,2180-07-23 12:00:00,2180-07-23 15:00:00,0.0,94.000000,88.000000,56.00,64.000000,23.000000,97.000000,52.0,39.400002
5,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 16:00:00,0.0,105.000000,91.250000,51.75,61.750000,21.000000,94.000000,52.0,39.400002
6,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 16:01:00,0.0,105.000000,91.000000,55.00,64.000000,21.000000,94.000000,52.0,39.400002
7,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 17:00:00,0.0,97.000000,95.000000,58.00,67.000000,20.000000,95.000000,52.0,39.400002
8,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 18:00:00,0.0,100.000000,86.000000,53.00,60.000000,21.000000,95.000000,52.0,39.400002
9,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 19:00:00,0.0,97.000000,93.000000,41.00,56.000000,16.000000,98.000000,52.0,39.400002


In [51]:
dataset_final['janela_index'].value_counts().head(20)

janela_index
2     259935
1     255528
3     246978
4     239182
5     230186
6     220097
7     205778
0     205565
8     190997
9     177862
10    168798
11    158160
12    145973
13    135656
14    130049
15    122906
16    117386
17    111532
18    104490
19     99103
Name: count, dtype: int64

In [40]:
dataset_final.to_parquet('mimic_tratada')

In [15]:
# Explorando a estrutura dos dados
print("Shape do dataset:", dataset.shape)
print("\nColunas:", dataset.columns.tolist())
print("\nTipos de dados:")
print(dataset.dtypes)

Shape do dataset: (7002766, 16)

Colunas: ['subject_id', 'stay_id', 'janela_index', 'inicio_janela', 'charttime', 'tem_sepse', 'fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'ida', 'pes', 'alt']

Tipos de dados:
subject_id                int64
stay_id                   int64
janela_index              int64
inicio_janela    datetime64[ns]
charttime        datetime64[ns]
tem_sepse               float64
fc                      float64
pas                     float64
pad                     float64
pam                     float64
fr                      float64
spo                     float64
tem                     float64
ida                     float64
pes                     float64
alt                     float64
dtype: object


In [ ]:
# Analisando colunas específicas
print("\nVisualizando valores únicos para id do paciente e da internação:")
for col in dataset.columns[:2]:  # Mostra só as 2 primeiras para não sobrecarregar
    if col in dataset.columns:
        print(f"{col}: {dataset[col].nunique()} valores únicos")
        print(f"  Exemplo: {dataset[col].head(1).tolist()}")

print(f"\nTotal de linhas: {len(dataset_orig)}")
print(f"Total de colunas: {len(dataset.columns)}")


Visualizando valores únicos para id do paciente e da internação:
subject_id: 48144 valores únicos
  Exemplo: [10000032]
stay_id: 48144 valores únicos
  Exemplo: [39553978]

Total de linhas: 7002766
Total de colunas: 16


- É importante perceber que o número de valores únicos para o ID do paciente e para o ID da estadia é o mesmo, pois a base foi criada utilizando apenas a primeira estadia de cada paciente.

## 3. Serialização Temporal dos dados

In [ ]:
# Ordena os daddos por paciente -> internação -> início da janela. Criando sequências temporais ordenadas para cada paciente
df_serializado_4h = dataset_final.sort_values(['subject_id', 'stay_id', 'inicio_janela'])

# Vamos filtrar os dados para manter apenas as janelas de 0-9 (10 janelas temporais)
df_filtrado_10j = df_serializado_4h.groupby(['subject_id', 'stay_id']).filter(lambda x: x['janela_index'].nunique() > 0)
df_filtrado_10j = df_filtrado_10j[df_filtrado_10j['janela_index'] < 10].reset_index(drop=True) # mudar para 10 janelas depois

In [5]:
df_filtrado_10j.groupby(['subject_id', 'stay_id'])['janela_index'].nunique().describe()

count    47290.000000
mean        13.213724
std          5.211022
min          1.000000
25%          9.000000
50%         13.000000
75%         19.000000
max         20.000000
Name: janela_index, dtype: float64

In [38]:
print(df_filtrado_10j.loc[df_filtrado_10j['janela_index'] > 9, 'janela_index'].unique())

[]


#### Resultados dos testes:
- 48.144 internações mantidas
- Média de 7.9 janelas por paciente
- 75% tem 10 janelas completas
- Nenhuma janela > 9 (filtragem funcionou)

## 4. Transformando em formato X, y, mask

In [7]:
# Identificar colunas de features (excluindo colunas de identificação)
print("Colunas disponíveis:")
print(df_filtrado_10j.columns.tolist())

# Definir colunas que NÃO são features (identificadores)
colunas_id = ['subject_id', 'stay_id', 'janela_index', 'charttime', 'inicio_janela']
# Definir coluna target
target_col = 'tem_sepse'

# Colunas de features (exames)
feature_cols = [col for col in df_filtrado_10j.columns 
                if col not in colunas_id + [target_col]]

print(f"\nColunas de features (exames): {len(feature_cols)}")
print(feature_cols)

Colunas disponíveis:
['subject_id', 'stay_id', 'janela_index', 'inicio_janela', 'charttime', 'tem_sepse', 'fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'ida', 'pes']

Colunas de features (exames): 8
['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'ida', 'pes']


In [ ]:
# Criar lista única de pacientes/internações
print("Criando lista de pacientes únicos...")
pacientes_unicos = df_filtrado_10j.groupby(['subject_id', 'stay_id']).first().reset_index()
print(f"Total de pacientes/internações únicos: {len(pacientes_unicos)}")

# Definir dimensões
n_pacientes = len(pacientes_unicos)
n_janelas = 10  # máximo de janelas
n_features = len(feature_cols)

print(f"Dimensões do array X: ({n_pacientes}, {n_janelas}, {n_features})")

# Inicializar arrays
X = np.zeros((n_pacientes, n_janelas, n_features))
y = np.zeros(n_pacientes)
mask = np.zeros((n_pacientes, n_janelas))

print("Arrays inicializados. Preenchendo dados...")

Criando lista de pacientes únicos...
Total de pacientes/internações únicos: 47290
Dimensões do array X: (47290, 20, 8)
Arrays inicializados. Preenchendo dados...


### Explicação

- **Agrupamento único:** Primeiro, agrupamos o DataFrame por `subject_id` e `stay_id` para garantir que cada paciente/internação seja tratado como uma sequência independente. Isso é essencial para não misturar dados de diferentes pacientes no LSTM.

- **Definição das dimensões:** Calculamos:
  - `n_pacientes`: total de pacientes/internações únicos (cada um será uma sequência para o LSTM)
  - `n_janelas`: número máximo de janelas temporais por paciente (aqui, 10)
  - `n_features`: quantidade de variáveis clínicas usadas como entrada

- **Inicialização dos arrays:**
  - `X`: array 3D que vai guardar os dados de entrada no formato [pacientes, janelas, features]
  - `y`: array 1D com tamanho igual ao número de pacientes, onde cada posição armazena o alvo (target) daquele paciente (ex: se teve sepse ou não)
  - `mask`: array 2D que indica quais janelas de cada paciente realmente têm dados válidos (útil para lidar com sequências de tamanhos diferentes)

- **Resumo:** Esta célula prepara toda a estrutura de dados para que o LSTM possa receber as sequências de cada paciente, sabendo exatamente onde estão os dados válidos e qual é o alvo de cada caso.

### Versão sem tratamento

In [9]:
# Preencher arrays
for i, (_, paciente_row) in enumerate(pacientes_unicos.iterrows()):
    if i % 5000 == 0:
        print(f"Processando paciente {i}/{n_pacientes}...")
    
    subject_id = paciente_row['subject_id']
    stay_id = paciente_row['stay_id']
    
    # Obter dados deste paciente/internação
    dados_paciente = df_filtrado_10j[
        (df_filtrado_10j['subject_id'] == subject_id) & 
        (df_filtrado_10j['stay_id'] == stay_id)
    ].sort_values('janela_index')
    
    # Preencher target y (usar o valor da primeira linha, pois deve ser igual para todas as janelas)
    y[i] = dados_paciente['tem_sepse'].iloc[0]
    
    # Preencher features X e mask
    for j, (_, janela_row) in enumerate(dados_paciente.iterrows()):
        if j < n_janelas:  # garantir que não excede 10 janelas
            janela_idx = int(janela_row['janela_index'])
            if janela_idx < n_janelas:
                # Preencher features
                X[i, janela_idx, :] = janela_row[feature_cols].values
                # Marcar como válido na mask
                mask[i, janela_idx] = 1

print("✅ Preenchimento concluído!")
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print(f"Shape mask: {mask.shape}")

Processando paciente 0/47290...
Processando paciente 5000/47290...
Processando paciente 10000/47290...
Processando paciente 15000/47290...
Processando paciente 20000/47290...
Processando paciente 25000/47290...
Processando paciente 30000/47290...
Processando paciente 35000/47290...
Processando paciente 40000/47290...
Processando paciente 45000/47290...
✅ Preenchimento concluído!
Shape X: (47290, 20, 8)
Shape y: (47290,)
Shape mask: (47290, 20)


### Explicação

- **Objetivo:** Esta etapa preenche os arrays `X`, `y` e `mask` com os dados de cada paciente/internação, organizando-os no formato necessário para treinar o LSTM.

- **Como funciona:**
  - O loop percorre todos os pacientes/internações únicos.
  - Para cada paciente:
    - Seleciona todas as janelas temporais disponíveis, já ordenadas.
    - O alvo (`y`) é preenchido com o valor da variável de interesse (ex: presença de sepse), que é igual para todas as janelas daquele paciente.
    - Para cada janela temporal:
      - Os valores das variáveis clínicas (features) são inseridos na posição correta do array `X`.
      - A matriz `mask` é atualizada para indicar quais posições de `X` contêm dados válidos (1 = dado presente, 0 = ausente).
    - O código garante que não sejam excedidas as 10 janelas por paciente, respeitando o formato [pacientes, janelas, features].

- **Por que isso é importante?**
  - O LSTM exige que os dados estejam organizados como sequências temporais de tamanho fixo, mesmo que alguns pacientes tenham menos janelas. A máscara (`mask`) permite que o modelo ignore posições "vazias" durante o treinamento.
  - Esse preenchimento transforma a tabela original em um formato tridimensional, pronto para ser usado em redes neurais recorrentes.

- **Resumo:** Após esta célula, os arrays `X`, `y` e `mask` estarão completos, representando cada paciente como uma sequência temporal de exames, com o respectivo alvo e indicação de dados válidos.

### Verificação da versão escolhida

In [10]:
# Verificar os dados criados
print("📊 VERIFICAÇÃO DOS DADOS CRIADOS:")
print(f"Total de pacientes: {len(y)}")
print(f"Com sepse: {np.sum(y == 1)}")
print(f"Sem sepse: {np.sum(y == 0)}")
print(f"Proporção com sepse: {np.mean(y):.3f}")

print(f"\n📈 ESTATÍSTICAS DAS SEQUÊNCIAS:")
janelas_por_paciente = np.sum(mask, axis=1)
print(f"Média de janelas por paciente: {np.mean(janelas_por_paciente):.2f}")
print(f"Min janelas: {np.min(janelas_por_paciente)}")
print(f"Max janelas: {np.max(janelas_por_paciente)}")

print(f"\n🔍 VERIFICAÇÃO DE VALORES:")
print(f"X - Min: {np.min(X):.2f}, Max: {np.max(X):.2f}")
print(f"Valores NaN em X: {np.sum(np.isnan(X))}")
print(f"Valores NaN em y: {np.sum(np.isnan(y))}")

# Salvar no formato .npz (igual ao notebook FASE 5)
caminho_arquivo = "D:/HD_Externo/TEDAS/mimic_X_y_mask_20j.npz"
np.savez_compressed(
    caminho_arquivo,
    X=X,
    y=y,
    mask=mask
)

print(f"\n💾 Arquivo salvo: {caminho_arquivo}")

📊 VERIFICAÇÃO DOS DADOS CRIADOS:
Total de pacientes: 47290
Com sepse: 6688
Sem sepse: 40602
Proporção com sepse: 0.141

📈 ESTATÍSTICAS DAS SEQUÊNCIAS:
Média de janelas por paciente: 4.08
Min janelas: 1.0
Max janelas: 16.0

🔍 VERIFICAÇÃO DE VALORES:
X - Min: 0.00, Max: 250.00
Valores NaN em X: 0
Valores NaN em y: 0

💾 Arquivo salvo: D:/HD_Externo/TEDAS/mimic_X_y_mask_20j.npz


### Balanceamento antigo

In [52]:
# 📥 Carregar dados normalizados
data = np.load("D:\HD_Externo\LSTM_apli_MIMIC\mimic_X_y_mask_10j.npz")
X = data['X']
y = data['y']
mask = data['mask']

# 🎯 Encontrar índices das classes
idx_sepse = np.where(y == 1)[0]
idx_nsepse = np.where(y == 0)[0]

# ⚖️ Balancear: mesmo número das duas classes
n = min(len(idx_sepse), len(idx_nsepse))
idx_balanced = np.concatenate([idx_sepse[:n], idx_nsepse[:n]])
np.random.shuffle(idx_balanced)

# 📦 Selecionar subconjunto balanceado
X_bal = X[idx_balanced]
y_bal = y[idx_balanced]
mask_bal = mask[idx_balanced]

# 💾 Salvar arquivo balanceado
np.savez_compressed(
    "D:\HD_Externo\LSTM_apli_MIMIC\mimic_X_y_mask_10j_clean.npz",
    X=X_bal,
    y=y_bal,
    mask=mask_bal
)

print("✅ Arquivo balanceado salvo com sucesso.")

✅ Arquivo balanceado salvo com sucesso.


In [53]:

# 📍 Caminhos dos arquivos
arquivo_original = "D:\HD_Externo\LSTM_apli_MIMIC\mimic_X_y_mask_10j.npz"
arquivo_balanceado = "D:\HD_Externo\LSTM_apli_MIMIC\mimic_X_y_mask_10j_clean.npz"

# 📥 Carregar os arquivos
dados_orig = np.load(arquivo_original)
dados_bal = np.load(arquivo_balanceado)

# 🎯 Extrair os rótulos
y_orig = dados_orig["y"]
y_bal = dados_bal["y"]

# 📊 Estatísticas antes do balanceamento
com_sepse_antes = np.sum(y_orig == 1)
sem_sepse_antes = np.sum(y_orig == 0)
total_antes = len(y_orig)

# 📊 Estatísticas depois do balanceamento
com_sepse_depois = np.sum(y_bal == 1) 
sem_sepse_depois = np.sum(y_bal == 0)
total_depois = len(y_bal)

# 🖨️ Resultados
print("📊 ANTES DO BALANCEAMENTO:")
print(f"✔️ Com Sepse: {com_sepse_antes}")
print(f"✔️ Sem Sepse: {sem_sepse_antes}")
print(f"🔢 Total: {total_antes}\n")

print("📊 DEPOIS DO BALANCEAMENTO:")
print(f"✔️ Com Sepse: {com_sepse_depois}")
print(f"✔️ Sem Sepse: {sem_sepse_depois}")
print(f"🔢 Total: {total_depois}")


📊 ANTES DO BALANCEAMENTO:
✔️ Com Sepse: 6688
✔️ Sem Sepse: 40602
🔢 Total: 47290

📊 DEPOIS DO BALANCEAMENTO:
✔️ Com Sepse: 6688
✔️ Sem Sepse: 6688
🔢 Total: 13376


## 5. Análise dos dados

In [ ]:
# 🔍 ANÁLISE DETALHADA: DADOS ANTES vs DEPOIS DA NORMALIZAÇÃO

# Carregar dados originais (sem normalização)
dados_orig = np.load("D:/HD_Externo/TEDAS/mimic_X_y_mask_10j_clean.npz")
X_orig = dados_orig['X']

# Carregar dados normalizados
dados_norm = np.load("D:/HD_Externo/TEDAS/mimic_X_y_mask_10j_clean_norm.npz")
X_norm = dados_norm['X']

print("📊 COMPARAÇÃO ANTES vs DEPOIS DA NORMALIZAÇÃO:")
print("=" * 60)

# Analisar cada feature
feature_names = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'ida', 'pes', 'alt']

for i, nome in enumerate(feature_names):
    # Dados válidos (onde mask = 1)
    mask_flat = mask.reshape(-1)
    
    orig_values = X_orig[:, :, i].reshape(-1)[mask_flat == 1]
    norm_values = X_norm[:, :, i].reshape(-1)[mask_flat == 1]
    
    print(f"\n🔹 {nome.upper()}:")
    print(f"   Original: Min={orig_values.min():.2f}, Max={orig_values.max():.2f}, "
          f"Std={orig_values.std():.2f}")
    print(f"   Normalizado: Min={norm_values.min():.3f}, Max={norm_values.max():.3f}, "
          f"Std={norm_values.std():.3f}")
    
    # Verificar se houve mudança significativa
    if orig_values.std() > 0:
        reducao_std = (1 - norm_values.std() / orig_values.std()) * 100
        print(f"   📉 Redução do desvio padrão: {reducao_std:.1f}%")

print("\n" + "=" * 60)
print("💡 INTERPRETAÇÃO:")
print("- Se os valores originais já tinham escalas similares, a normalização tem pouco impacto")
print("- Se algumas features tinham std muito baixo, a normalização pode 'achatar' dados importantes")
print("- Para LSTM, normalização geralmente ajuda, mas nem sempre é dramática")

📊 COMPARAÇÃO ANTES vs DEPOIS DA NORMALIZAÇÃO:

🔹 FC:
   Original: Min=0.00, Max=212.00, Std=42.27
   Normalizado: Min=0.000, Max=1.000, Std=0.199
   📉 Redução do desvio padrão: 99.5%

🔹 PAS:
   Original: Min=0.00, Max=234.00, Std=57.63
   Normalizado: Min=0.000, Max=1.000, Std=0.246
   📉 Redução do desvio padrão: 99.6%

🔹 PAD:
   Original: Min=0.00, Max=150.00, Std=32.35
   Normalizado: Min=0.000, Max=1.000, Std=0.216
   📉 Redução do desvio padrão: 99.3%

🔹 PAM:
   Original: Min=0.00, Max=170.00, Std=38.20
   Normalizado: Min=0.000, Max=1.000, Std=0.225
   📉 Redução do desvio padrão: 99.4%

🔹 FR:
   Original: Min=0.00, Max=60.00, Std=9.64
   Normalizado: Min=0.000, Max=1.000, Std=0.161
   📉 Redução do desvio padrão: 98.3%

🔹 SPO:
   Original: Min=0.00, Max=100.00, Std=47.18
   Normalizado: Min=0.000, Max=1.000, Std=0.472
   📉 Redução do desvio padrão: 99.0%

🔹 TEM:
   Original: Min=0.00, Max=42.30, Std=3.87
   Normalizado: Min=0.000, Max=1.000, Std=0.092
   📉 Redução do desvio padrão: 

In [ ]:
# 🎯 ANÁLISE DAS ESCALAS ORIGINAIS

print("🔬 ANÁLISE DAS ESCALAS ORIGINAIS:")
print("=" * 50)

feature_names = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'ida', 'pes', 'alt']

escalas_diferentes = []
for i, nome in enumerate(feature_names):
    mask_flat = mask.reshape(-1)
    orig_values = X_orig[:, :, i].reshape(-1)[mask_flat == 1]
    
    range_val = orig_values.max() - orig_values.min()
    escalas_diferentes.append((nome, range_val, orig_values.std()))

# Ordenar por range (escala)
escalas_diferentes.sort(key=lambda x: x[1], reverse=True)

print("Features ordenadas por escala (range):")
for nome, range_val, std_val in escalas_diferentes:
    print(f"  {nome:4s}: Range={range_val:8.2f}, Std={std_val:6.2f}")

# Calcular razão entre maior e menor escala
maior_range = escalas_diferentes[0][1]
menor_range = escalas_diferentes[-1][1]
razao_escalas = maior_range / menor_range if menor_range > 0 else 0

print(f"\n📊 Razão entre maior e menor escala: {razao_escalas:.1f}x")

if razao_escalas < 10:
    print("✅ Escalas já são relativamente similares - normalização tem impacto menor")
elif razao_escalas < 100:
    print("⚠️  Escalas moderadamente diferentes - normalização deveria ajudar um pouco")
else:
    print("🔥 Escalas muito diferentes - normalização deveria ajudar bastante")

print(f"\n💡 POSSÍVEIS MOTIVOS para resultados similares:")
print(f"1. Dados MIMIC já vêm pré-processados")
print(f"2. LSTM é robusto a diferenças de escala menores")
print(f"3. Outras técnicas (arquitetura, hiperparâmetros) têm mais impacto")
print(f"4. Dataset específico pode não se beneficiar muito da normalização")

🔬 ANÁLISE DAS ESCALAS ORIGINAIS:
Features ordenadas por escala (range):
  pes : Range=  250.00, Std= 26.12
  pas : Range=  234.00, Std= 57.63
  alt : Range=  218.00, Std= 85.13
  fc  : Range=  212.00, Std= 42.27
  pam : Range=  170.00, Std= 38.20
  pad : Range=  150.00, Std= 32.35
  spo : Range=  100.00, Std= 47.18
  fr  : Range=   60.00, Std=  9.64
  tem : Range=   42.30, Std=  3.87

📊 Razão entre maior e menor escala: 5.9x
✅ Escalas já são relativamente similares - normalização tem impacto menor

💡 POSSÍVEIS MOTIVOS para resultados similares:
1. Dados MIMIC já vêm pré-processados
2. LSTM é robusto a diferenças de escala menores
3. Outras técnicas (arquitetura, hiperparâmetros) têm mais impacto
4. Dataset específico pode não se beneficiar muito da normalização
